# Comparison of spectrum between eNATL60 outputs and Alitka Satellite : method gonzag on one track

Requisites :

 - git clone https://github.com/brodeau/gonzag
 
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag' ; 
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [3]:
import xarray as xr
import intake
import gcsfs
import os
import matplotlib.pyplot as plt

# With gonzag

In [4]:
from gonzag.config import IsZarr,ldebug, rmissval, l_save_track_on_model_grid

In [5]:
file_sat = '../results/results_Altika-eNATL60-Region1-fma/Altika-data-from-cloud-eNATL60-FMA-Region1-track0-sec.nc'
name_ssh_sat = 'sla_unfiltered'
file_mod = '../../Region01-SSH-hourly_2010-02-04-newt2014.nc'
name_ssh_mod = 'sossheig'
file_lsm_mod = '../../Region01-SSH-hourly_2010-02-04-newt2014.nc'
name_lsm_mod = '_FillValue'
l_griddist = True

In [6]:
!ls $file_sat

../results/results_Altika-eNATL60-Region1-fma/Altika-data-from-cloud-eNATL60-FMA-Region1-track0-sec.nc


In [7]:
(it1,it2), (Nts,Ntm) = gz.GetEpochTimeOverlap( file_sat , file_mod )
clsm = name_lsm_mod
if name_lsm_mod=='_FillValue': clsm = name_lsm_mod+'@'+name_ssh_mod
    

In [8]:
ModelGrid = gz.ModGrid( file_mod, it1, it2, file_lsm_mod, clsm, distorded_grid=l_griddist )


 *** what we use to define model land-sea mask:
    => "_FillValue@sossheig" in "../../Region01-SSH-hourly_2010-02-04-newt2014.nc"

 *** Computing angle distortion of the model grid ("-D" option invoked)...

 *** About model gridded (source) domain:
     * shape =  (763, 611)
     * horizontal resolution:  0.016657294601690575  degrees or  1.8507920031938399  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  281.87 292.17
     * lat_min, lat_max =  30.01 40.01
     * should we pay attention to possible STRONG local distorsion in the grid:  True
     * number of time records of interest for the interpolation to come:  2
       ==> time record indices: 46 to 47, included



In [9]:
SatelliteTrack = gz.SatTrack( file_sat, it1, it2, Np=Nts, \
                                  domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in ../results/results_Altika-eNATL60-Region1-fma/Altika-data-from-cloud-eNATL60-FMA-Region1-track0-sec.nc ...

 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  170
       ==> time record indices: 0 to 170, included



In [10]:
%time
RES = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat )

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
 *** "found" distance criterion when searching for nearest point on model grid is  1.3880940023953798  km

 *** Will use zoom boxes of width of 271 points for 1st attempts of nearest-point location...


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 1.3880940023953798 135 )
      +++ Treated point: 17/170 
          ==> Sat. coordinates:     30.994 288.851
          ==> Model nearest point:  30.997 288.854  ( 69 411 )
      +++ Treated point: 34/170 
          ==> Sat. coordinates:     31.977 288.579
          ==> Model nearest point:  31.978 288.582  ( 140 395 )
      +++ Treated point: 51/170 
          ==> Sat. coordinates:     32.959 288.302
          ==> Model nearest point:  32.957 288.308  ( 212 379 )
      +++ Treated point: 68/170 
          ==> Sat. coordinates:     33.941 288.021
          ==> Model nearest point:  33.947 288.015  ( 286 362 )
      +++ Treated point: 85/170 
     

In [11]:
print(ModelGrid.time)

[1.3913802e+09 1.3913838e+09]


In [12]:
print(ModelGrid.lat)

[[30.010681 30.010723 30.010763 ... 30.037752 30.037798 30.037844]
 [30.024738 30.02478  30.024822 ... 30.051926 30.051973 30.05202 ]
 [30.038794 30.038836 30.038877 ... 30.066097 30.066145 30.06619 ]
 ...
 [39.76764  39.76797  39.768303 ... 39.98261  39.982975 39.98334 ]
 [39.779232 39.779564 39.779896 ... 39.994583 39.994953 39.99532 ]
 [39.79082  39.791157 39.79149  ... 40.006557 40.006924 40.007294]]


In [13]:
print(ModelGrid.lon)

[[282.00876 282.02542 282.04208 ... 292.14117 292.15784 292.1745 ]
 [282.00873 282.0254  282.04205 ... 292.1411  292.15778 292.17444]
 [282.00867 282.02533 282.042   ... 292.14105 292.15775 292.1744 ]
 ...
 [281.86728 281.88388 281.9005  ... 291.98077 291.99744 292.0141 ]
 [281.86688 281.88348 281.90012 ... 291.98032 291.99698 292.01364]
 [281.8665  281.88312 281.89972 ... 291.97986 291.99652 292.01318]]


In [14]:
print(ModelGrid.domain_bounds)

[30.010681, 281.8665, 40.007294, 292.1745]


In [15]:
print(SatelliteTrack.time)

[1.39138285e+09 1.39138285e+09 1.39138286e+09 1.39138286e+09
 1.39138286e+09 1.39138286e+09 1.39138286e+09 1.39138286e+09
 1.39138286e+09 1.39138286e+09 1.39138286e+09 1.39138286e+09
 1.39138287e+09 1.39138287e+09 1.39138287e+09 1.39138287e+09
 1.39138287e+09 1.39138287e+09 1.39138287e+09 1.39138287e+09
 1.39138287e+09 1.39138287e+09 1.39138288e+09 1.39138288e+09
 1.39138288e+09 1.39138288e+09 1.39138288e+09 1.39138288e+09
 1.39138288e+09 1.39138288e+09 1.39138288e+09 1.39138288e+09
 1.39138289e+09 1.39138289e+09 1.39138289e+09 1.39138289e+09
 1.39138289e+09 1.39138289e+09 1.39138289e+09 1.39138289e+09
 1.39138289e+09 1.39138289e+09 1.39138290e+09 1.39138290e+09
 1.39138290e+09 1.39138290e+09 1.39138290e+09 1.39138290e+09
 1.39138290e+09 1.39138290e+09 1.39138290e+09 1.39138290e+09
 1.39138290e+09 1.39138291e+09 1.39138291e+09 1.39138291e+09
 1.39138291e+09 1.39138291e+09 1.39138291e+09 1.39138291e+09
 1.39138291e+09 1.39138291e+09 1.39138291e+09 1.39138292e+09
 1.39138292e+09 1.391382

In [16]:
print(SatelliteTrack.lat)

[30.068213 30.126069 30.183924 30.241777 30.299628 30.357478 30.415327
 30.473173 30.531019 30.588863 30.646705 30.704545 30.762384 30.820222
 30.878058 30.935892 30.993725 31.051556 31.109386 31.167214 31.22504
 31.282865 31.340688 31.398509 31.456329 31.514147 31.571963 31.629778
 31.687592 31.745403 31.803213 31.861021 31.918828 31.976633 32.034436
 32.092237 32.150037 32.207836 32.265632 32.323427 32.38122  32.439011
 32.496801 32.554589 32.612375 32.670159 32.727942 32.785723 32.843502
 32.90128  32.959056 33.01683  33.074603 33.132372 33.190141 33.247908
 33.305673 33.363437 33.421197 33.478957 33.536716 33.594472 33.652227
 33.709979 33.76773  33.825479 33.883226 33.940971 33.998715 34.056456
 34.114195 34.171933 34.229669 34.287403 34.345135 34.402866 34.460594
 34.518321 34.576046 34.633769 34.691489 34.749208 34.806925 34.86464
 34.922353 34.980065 35.037774 35.095482 35.153187 35.21089  35.268592
 35.326291 35.499378 35.55707  35.614759 35.672447 35.730132 35.787816
 35.8454

In [17]:
print(SatelliteTrack.lon)

[289.103209 289.087534 289.071844 289.05614  289.040423 289.024693
 289.008948 288.993191 288.97742  288.961636 288.945837 288.930022
 288.914195 288.898354 288.882498 288.866629 288.850746 288.834849
 288.818938 288.803012 288.78707  288.771115 288.755145 288.739161
 288.723163 288.707151 288.691124 288.675081 288.659023 288.64295
 288.626863 288.610762 288.594645 288.578514 288.562367 288.546206
 288.530028 288.513835 288.497627 288.481403 288.465165 288.448912
 288.432643 288.416359 288.400058 288.383741 288.367408 288.351061
 288.334697 288.318318 288.301923 288.285513 288.269086 288.252642
 288.236181 288.219705 288.203213 288.186705 288.170181 288.15364
 288.137083 288.120508 288.103916 288.087308 288.070683 288.054042
 288.037385 288.020711 288.00402  287.98731  287.970583 287.953839
 287.937079 287.920302 287.903507 287.886696 287.869867 287.853019
 287.836153 287.819271 287.80237  287.785452 287.768516 287.751564
 287.734593 287.717604 287.700595 287.683569 287.666524 287.6494

In [18]:
print(SatelliteTrack.size)

170


In [19]:
print(SatelliteTrack.keepit)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170]),)


In [20]:
print(RES.time)

[1.39138285e+09 1.39138285e+09 1.39138286e+09 1.39138286e+09
 1.39138286e+09 1.39138286e+09 1.39138286e+09 1.39138286e+09
 1.39138286e+09 1.39138286e+09 1.39138286e+09 1.39138286e+09
 1.39138287e+09 1.39138287e+09 1.39138287e+09 1.39138287e+09
 1.39138287e+09 1.39138287e+09 1.39138287e+09 1.39138287e+09
 1.39138287e+09 1.39138287e+09 1.39138288e+09 1.39138288e+09
 1.39138288e+09 1.39138288e+09 1.39138288e+09 1.39138288e+09
 1.39138288e+09 1.39138288e+09 1.39138288e+09 1.39138288e+09
 1.39138289e+09 1.39138289e+09 1.39138289e+09 1.39138289e+09
 1.39138289e+09 1.39138289e+09 1.39138289e+09 1.39138289e+09
 1.39138289e+09 1.39138289e+09 1.39138290e+09 1.39138290e+09
 1.39138290e+09 1.39138290e+09 1.39138290e+09 1.39138290e+09
 1.39138290e+09 1.39138290e+09 1.39138290e+09 1.39138290e+09
 1.39138290e+09 1.39138291e+09 1.39138291e+09 1.39138291e+09
 1.39138291e+09 1.39138291e+09 1.39138291e+09 1.39138291e+09
 1.39138291e+09 1.39138291e+09 1.39138291e+09 1.39138292e+09
 1.39138292e+09 1.391382

In [21]:
print(RES.lat)

[30.068213 30.126069 30.183924 30.241777 30.299628 30.357478 30.415327
 30.473173 30.531019 30.588863 30.646705 30.704545 30.762384 30.820222
 30.878058 30.935892 30.993725 31.051556 31.109386 31.167214 31.22504
 31.282865 31.340688 31.398509 31.456329 31.514147 31.571963 31.629778
 31.687592 31.745403 31.803213 31.861021 31.918828 31.976633 32.034436
 32.092237 32.150037 32.207836 32.265632 32.323427 32.38122  32.439011
 32.496801 32.554589 32.612375 32.670159 32.727942 32.785723 32.843502
 32.90128  32.959056 33.01683  33.074603 33.132372 33.190141 33.247908
 33.305673 33.363437 33.421197 33.478957 33.536716 33.594472 33.652227
 33.709979 33.76773  33.825479 33.883226 33.940971 33.998715 34.056456
 34.114195 34.171933 34.229669 34.287403 34.345135 34.402866 34.460594
 34.518321 34.576046 34.633769 34.691489 34.749208 34.806925 34.86464
 34.922353 34.980065 35.037774 35.095482 35.153187 35.21089  35.268592
 35.326291 35.499378 35.55707  35.614759 35.672447 35.730132 35.787816
 35.8454

In [22]:
print(RES.lon)

[289.103209 289.087534 289.071844 289.05614  289.040423 289.024693
 289.008948 288.993191 288.97742  288.961636 288.945837 288.930022
 288.914195 288.898354 288.882498 288.866629 288.850746 288.834849
 288.818938 288.803012 288.78707  288.771115 288.755145 288.739161
 288.723163 288.707151 288.691124 288.675081 288.659023 288.64295
 288.626863 288.610762 288.594645 288.578514 288.562367 288.546206
 288.530028 288.513835 288.497627 288.481403 288.465165 288.448912
 288.432643 288.416359 288.400058 288.383741 288.367408 288.351061
 288.334697 288.318318 288.301923 288.285513 288.269086 288.252642
 288.236181 288.219705 288.203213 288.186705 288.170181 288.15364
 288.137083 288.120508 288.103916 288.087308 288.070683 288.054042
 288.037385 288.020711 288.00402  287.98731  287.970583 287.953839
 287.937079 287.920302 287.903507 287.886696 287.869867 287.853019
 287.836153 287.819271 287.80237  287.785452 287.768516 287.751564
 287.734593 287.717604 287.700595 287.683569 287.666524 287.6494

In [23]:
print(RES.ssh_mod_np)

[ 0.61899459  0.62162822  0.62020218  0.61639076  0.60917342  0.59633201
  0.57384598  0.56341314  0.57152563  0.58070523  0.58691186  0.59786147
  0.60035282  0.59848857  0.59225827  0.58805448  0.58311564  0.57813799
  0.58007604  0.59020782  0.5964765   0.59688139  0.59306985  0.58476168
  0.56867456  0.55020565  0.53738981  0.52699113  0.51762837  0.51586342
  0.51535225  0.51217234  0.50663471  0.50596762  0.50673968  0.50232905
  0.50108778  0.50712931  0.51751059  0.52978635  0.54079831  0.55306077
  0.56732976  0.58965319  0.61458576  0.63425022  0.64449996  0.64750177
  0.64975154  0.65027112  0.647928    0.6437732   0.63414323  0.62704539
  0.62318015  0.62093818  0.61963522  0.62482882  0.62570995  0.61394036
  0.6191572   0.63414448  0.64651036  0.65421665  0.65452492  0.65479279
  0.65402281  0.64339054  0.63616937  0.63123691  0.62577993  0.62221098
  0.6209619   0.62233371  0.62178981  0.61828989  0.61638212  0.61924893
  0.62435871  0.62796944  0.62930095  0.6289165   0

In [24]:
print(RES.ssh_mod)

[ 0.61943584  0.62193331  0.62033978  0.6163177   0.60879109  0.59525745
  0.57181361  0.563154    0.57082026  0.58048617  0.58697974  0.59792772
  0.60021342  0.59818525  0.59165347  0.58818157  0.5838046   0.57859661
  0.58016379  0.5907443   0.59678271  0.59710781  0.59321415  0.58460547
  0.56763647  0.54853521  0.53536375  0.52681626  0.51887143  0.51606871
  0.5152881   0.51175703  0.50734065  0.50639603  0.50697134  0.50221131
  0.50146779  0.50676973  0.51720411  0.52967876  0.54085828  0.55177701
  0.56620052  0.58903219  0.61452599  0.63186948  0.64232784  0.6496318
  0.65129726  0.6510996   0.6486928   0.64365114  0.63526105  0.62735392
  0.62323542  0.62154628  0.61976176  0.62525505  0.62683738  0.61432034
  0.619305    0.63249074  0.64577653  0.65397701  0.65421966  0.6543369
  0.65341262  0.64458315  0.63591181  0.63009843  0.62526619  0.62140244
  0.62167712  0.62304095  0.62224048  0.61943426  0.61698097  0.62018687
  0.62499974  0.62827053  0.62915756  0.62848191  0.6

In [25]:
print(RES.ssh_sat)

[ 0.059  0.087  0.087  0.091  0.105  0.055  0.077  0.109  0.127  0.123
  0.15   0.118  0.129  0.127  0.122  0.114  0.1    0.093  0.104  0.061
  0.025  0.026  0.023  0.014 -0.012 -0.028 -0.027 -0.033 -0.057 -0.056
 -0.075 -0.069 -0.091 -0.097 -0.108 -0.113 -0.121 -0.132 -0.129 -0.145
 -0.137 -0.121 -0.136 -0.137 -0.116 -0.101 -0.093 -0.076 -0.08  -0.058
 -0.062 -0.051 -0.057 -0.053 -0.038 -0.025 -0.038 -0.032 -0.032 -0.048
 -0.053 -0.048 -0.047 -0.052 -0.053 -0.068 -0.069 -0.067 -0.069 -0.062
 -0.072 -0.065 -0.047 -0.051 -0.057 -0.05  -0.044 -0.036 -0.019 -0.003
  0.018  0.049  0.044  0.038  0.065  0.082  0.084  0.107  0.117  0.157
  0.219  0.278  0.27   0.289  0.294  0.305  0.333  0.37   0.369  0.35
  0.333  0.33   0.35   0.337  0.337  0.359  0.374  0.417  0.458  0.496
  0.564  0.619  0.646  0.655  0.634  0.618  0.596  0.554  0.49   0.422
  0.331  0.212  0.116  0.08   0.034 -0.009 -0.016  0.023  0.035  0.022
  0.029  0.049  0.058  0.061  0.063  0.062  0.051  0.038  0.031  0.01
 -0.011 

In [26]:
print(RES.distance)

[   0.            6.60952852   13.21905698   19.82845445   26.43769828
   33.04689608   39.65609103   46.26500008   52.87399124   59.48282533
   66.09154548   72.70017288   79.30872749   85.91725242   92.52566055
   99.13390745  105.74212253  112.35019682  118.95823788  125.56615858
  132.17398002  138.78174424  145.38938593  151.99688253  158.60434159
  165.21165414  171.81884125  178.42603221  185.03320449  191.64014089
  198.24703536  204.85377895  211.46052266  218.06711403  224.67359579
  231.27992373  237.8862705   244.4925919   251.09867074  257.7047444
  264.31066052  270.91644004  277.5222121   284.12784605  290.73338446
  297.33880491  303.94421479  310.54946185  317.15461026  323.75972415
  330.36471619  336.96956398  343.57441816  350.1789534   356.78360161
  363.38810245  369.99247673  376.59683182  383.20084245  389.80496204
  396.40906005  403.01296248  409.61686326  416.22052374  422.82418095
  429.42770441  436.03109332  442.63436837  449.23763688  455.84072475
  462.4

In [27]:
MG=ModelGrid
ST=SatelliteTrack


In [28]:
import time
from gonzag.ncio   import GetModel2DVar, GetSatSSH
from gonzag.config import IsZarr, ldebug, ivrb, nb_talk, l_plot_meshes, deg2km, rfactor, search_box_w_km, l_save_track_on_model_grid, l_plot_meshes, rmissval
from gonzag.utils  import *
from gonzag.bilin_mapping import BilinTrack


In [29]:
(Nj,Ni) = MG.shape
d_found_km = rfactor*MG.HResDeg*deg2km
np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
Nt = ST.size
if_talk = Nt//nb_talk
startTime = time.time()

In [30]:
print(d_found_km,np_box_radius,Nt)

1.3880940023953798 135 170


In [31]:
BT = BilinTrack( ST.lat, ST.lon, MG.lat, MG.lon, src_grid_local_angle=MG.xangle, \
                         k_ew_per=MG.EWPer, rd_found_km=d_found_km, np_box_r=np_box_radius, freq_talk=if_talk )


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 1.3880940023953798 135 )
      +++ Treated point: 17/170 
          ==> Sat. coordinates:     30.994 288.851
          ==> Model nearest point:  30.997 288.854  ( 69 411 )
      +++ Treated point: 34/170 
          ==> Sat. coordinates:     31.977 288.579
          ==> Model nearest point:  31.978 288.582  ( 140 395 )
      +++ Treated point: 51/170 
          ==> Sat. coordinates:     32.959 288.302
          ==> Model nearest point:  32.957 288.308  ( 212 379 )
      +++ Treated point: 68/170 
          ==> Sat. coordinates:     33.941 288.021
          ==> Model nearest point:  33.947 288.015  ( 286 362 )
      +++ Treated point: 85/170 
          ==> Sat. coordinates:     34.922 287.735
          ==> Model nearest point:  34.918 287.737  ( 360 346 )
      +++ Treated point: 102/170 
          ==> Sat. coordinates:     36.019 287.409
          ==> Model nearest point:  36.025 287.403  ( 446 327 )
      +

In [32]:
for jt in range(Nt):
    [jj,ji] = BT.NP[jt,:]
    if (jj,ji) == (-1,-1):
        print('      ===> NO! Was not found!')
    else:
        PlotMesh( (ST.lon[jt],ST.lat[jt]), MG.lat, MG.lon, BT.SM[jt,:,:], BT.WB[jt,:], \
                                  fig_name='../plots/debug/gonzag_mesh_jt'+'%5.5i'%(jt)+'.png' )


      ===> NO! Was not found!
      ===> NO! Was not found!


In [33]:
from math import radians, cos, sin, asin, sqrt, pi, tan, log, atan2, copysign

import numpy as nmp

from gonzag.config  import ldebug, ivrb
from gonzag.utils   import Haversine, find_j_i_min, degE_to_degWE



In [34]:
Yt=ST.lat
Xt=ST.lon
Ys=MG.lat
Xs= MG.lon
src_grid_local_angle=MG.xangle
k_ew_per=MG.EWPer 
rd_found_km=d_found_km
np_box_r=np_box_radius
freq_talk=if_talk

In [35]:
    def nrpt( self ):
        xnp = nmp.zeros((self.Nt,2), dtype=nmp.int64)
        [jj,ji] = [self.nprad,self.nprad] ; # stupid first guess here...
        for jt in range(self.Nt):
            ltalk = ((jt+1)%self.ftalk==0)
            [jj,ji] = NearestPoint( (self.Yt[jt],self.Xt[jt]), self.Ys, self.Xs, \
                                    rd_found_km=self.rfound, j_prv=jj, i_prv=ji, np_box_r=self.nprad )
            if ji==0 or ji==self.Ni-1 and self.kewp == -1: [jj,ji] = [-1,-1]
            if jj==0 or jj==self.Nj-1:                     [jj,ji] = [-1,-1]
            
            xnp[jt,:] = [jj,ji]
        return xnp


In [36]:
    def srcm( self ):
        xsp = nmp.zeros((self.Nt,4,2), dtype=nmp.int64)

        for jt in range(self.Nt):
            [jP,iP] = self.NP[jt,:]
            if [jP,iP] != [-1,-1]:
                angle = 0.
                if nmp.shape(self.sangle) == nmp.shape(self.Ys): angle = self.sangle[jP,iP]
                xsp[jt,:,:] = IDSourceMesh( (self.Yt[jt],self.Xt[jt]), self.Ys, self.Xs, jP, iP, \
                                            k_ew_per=self.kewp, grid_s_angle=angle )
        return xsp

    def wght( self ):
        x4w = nmp.zeros((self.Nt,4))
        for jt in range(self.Nt):
            x4w[jt,:] = WeightBL( (self.Yt[jt],self.Xt[jt]), self.Ys, self.Xs, self.SM[jt,:,:] )
        return x4w


In [37]:
(Nt,) = nmp.shape(Yt)

sangle = src_grid_local_angle
kewp   = k_ew_per
rfound = rd_found_km
nprad  = np_box_r
(Nj,Ni) = Ys.shape
ftalk = 2*Nt   
NP = nmp.zeros((Nt,2)  , dtype=nmp.int64)

In [38]:
from gonzag.bilin_mapping import *

In [39]:
        xnp = nmp.zeros((Nt,2), dtype=nmp.int64)
        [jj,ji] = [nprad,nprad] ; # stupid first guess here...
        for jt in range(Nt):
            ltalk = ((jt+1)%ftalk==0)
            [jj,ji] = NearestPoint( (Yt[jt],Xt[jt]), Ys, Xs, \
                                    rd_found_km=rfound, j_prv=jj, i_prv=ji, np_box_r=nprad )
            if ji==0 or ji==Ni-1 and kewp == -1: [jj,ji] = [-1,-1]
            if jj==0 or jj==Nj-1:                     [jj,ji] = [-1,-1]
            
            xnp[jt,:] = [jj,ji]


In [40]:
print(xnp)

[[  3 426]
 [  7 425]
 [ 11 424]
 [ 15 423]
 [ 19 422]
 [ 23 421]
 [ 27 420]
 [ 32 419]
 [ 36 418]
 [ 40 417]
 [ 44 416]
 [ 48 415]
 [ 52 415]
 [ 56 414]
 [ 60 413]
 [ 65 412]
 [ 69 411]
 [ 73 410]
 [ 77 409]
 [ 81 408]
 [ 85 407]
 [ 90 406]
 [ 94 405]
 [ 98 404]
 [102 403]
 [106 402]
 [110 401]
 [115 400]
 [119 400]
 [123 399]
 [127 398]
 [131 397]
 [136 396]
 [140 395]
 [144 394]
 [148 393]
 [153 392]
 [157 391]
 [161 390]
 [165 389]
 [169 388]
 [174 387]
 [178 386]
 [182 385]
 [186 384]
 [191 383]
 [195 382]
 [199 382]
 [204 381]
 [208 380]
 [212 379]
 [216 378]
 [221 377]
 [225 376]
 [229 375]
 [234 374]
 [238 373]
 [242 372]
 [247 371]
 [251 370]
 [255 369]
 [260 368]
 [264 367]
 [268 366]
 [273 365]
 [277 364]
 [281 363]
 [286 362]
 [290 361]
 [294 360]
 [299 359]
 [303 358]
 [307 358]
 [312 357]
 [316 356]
 [321 355]
 [325 354]
 [329 353]
 [334 352]
 [338 351]
 [343 350]
 [347 349]
 [351 348]
 [356 347]
 [360 346]
 [365 345]
 [369 344]
 [374 343]
 [378 342]
 [383 341]
 [387 340]

In [41]:
NP=xnp

In [42]:
        xsp = nmp.zeros((Nt,4,2), dtype=nmp.int64)

        for jt in range(Nt):
            [jP,iP] = NP[jt,:]
            if [jP,iP] != [-1,-1]:
                angle = 0.
                if nmp.shape(sangle) == nmp.shape(Ys): angle = sangle[jP,iP]
                #
                xsp[jt,:,:] = IDSourceMesh( (Yt[jt],Xt[jt]), Ys, Xs, jP, iP, \
                                            k_ew_per=kewp, grid_s_angle=angle )


In [43]:
print(xsp)

[[[  3 426]
  [  3 425]
  [  2 425]
  [  2 426]]

 [[  7 425]
  [  7 424]
  [  6 424]
  [  6 425]]

 [[ 11 424]
  [ 11 423]
  [ 10 423]
  [ 10 424]]

 ...

 [[757 262]
  [757 261]
  [756 261]
  [756 262]]

 [[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]]

 [[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]]]


In [44]:
SM=xsp

In [45]:
        x4w = nmp.zeros((Nt,4))
        #
        for jt in range(Nt):
            x4w[jt,:] = WeightBL( (Yt[jt],Xt[jt]), Ys, Xs, SM[jt,:,:] )
        #


In [46]:
x4w


array([[5.40091964e-01, 2.07510822e-01, 7.00574613e-02, 1.82339753e-01],
       [6.72770226e-01, 1.73604741e-01, 3.15108967e-02, 1.22114137e-01],
       [8.21780913e-01, 1.26666175e-01, 6.88494937e-03, 4.46679627e-02],
       [8.86659648e-01, 5.05259655e-02, 3.38647701e-03, 5.94279097e-02],
       [8.29676831e-01, 7.52098935e-03, 1.46253780e-03, 1.61339642e-01],
       [6.68140583e-01, 5.66715467e-02, 2.15163649e-02, 2.53671505e-01],
       [5.19156094e-01, 8.99084053e-02, 5.77088099e-02, 3.33226691e-01],
       [3.99917358e-01, 3.84143437e-01, 1.05797445e-01, 1.10141761e-01],
       [4.52689841e-01, 2.62760658e-01, 1.04505363e-01, 1.80044138e-01],
       [4.91468267e-01, 1.56139344e-01, 8.49624300e-02, 2.67429959e-01],
       [5.15968334e-01, 6.46836456e-02, 4.67146582e-02, 3.72633363e-01],
       [5.01677277e-01, 4.76608522e-01, 1.05788856e-02, 1.11353160e-02],
       [4.64964655e-01, 8.75236129e-02, 7.08935300e-02, 3.76618202e-01],
       [4.34024796e-01, 1.84768356e-01, 1.13826345e

In [47]:
WB=x4w

In [63]:
jt=Nt-1
[jj,ji] = NP[jt,:]
print(ST.lon[jt],ST.lat[jt], SM[jt,:,:], WB[jt,:])

286.181521 39.935376 [[0 0]
 [0 0]
 [0 0]
 [0 0]] [0.25 0.25 0.25 0.25]


In [49]:
jt=1
[jj,ji] = NP[jt,:]
print(ST.lon[jt],ST.lat[jt], SM[jt,:,:], WB[jt,:])

289.087534 30.126069 [[  7 425]
 [  7 424]
 [  6 424]
 [  6 425]] [0.67277023 0.17360474 0.0315109  0.12211414]


In [64]:
        vssh_m_np = nmp.zeros(Nt) ; vssh_m_np[:] = rmissval; # vector to store the model data interpolated in time and space (nearest-point) on the satellite track...
        vssh_m_bl = nmp.zeros(Nt) ; vssh_m_bl[:] = rmissval; # vector to store the model data interpolated in time and space (bilinear) on the satellite track...
        vdistance = nmp.zeros(Nt)
    
        # Time increment on the satellite time:
        ktm1   = 0   ; ktm2   = 0
        ktm1_o = -10 ; ktm2_o = -10


In [65]:
jt=Nt-1
itt = ST.time[jt]
kt = ktm1
while not (MG.time[kt]<=itt and MG.time[kt+1]>itt): kt=kt+1
print(itt,kt,MG.time[kt])

1391383021.56 0 1391380200.0


In [66]:
ktm1 = kt ; ktm2 = kt+1


In [67]:
            if (ktm1>ktm1_o) and (ktm2>ktm2_o):
                if (ktm1_o == -10) or (ktm1 > ktm2_o):
                    if ivrb>0: print(' *** Reading '+name_ssh_mod+' in '+MG.file+'\n    => at ktm1=', ktm1)
                    Xm1 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm1 )
                else:
                    Xm1[:,:] = Xm2[:,:]


In [68]:
fig=plt.figure(figsize=[7,5])
plt.pcolormesh(Xm1)
plt.colorbar()
plt.savefig('../plots/debug/plot_gonzag_Xm1.png')

In [69]:
Xm2 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm2 )

In [70]:
fig=plt.figure(figsize=[7,5])
plt.pcolormesh(Xm2)
plt.colorbar()
plt.savefig('../plots/debug/plot_gonzag_Xm2.png')

In [71]:
Xa = (Xm2 - Xm1) / float(MG.time[ktm2] - MG.time[ktm1])

In [72]:
print(float(MG.time[ktm2] - MG.time[ktm1]))

3600.0


In [73]:
fig=plt.figure(figsize=[7,5])
plt.pcolormesh(Xa)
plt.colorbar()
plt.savefig('../plots/debug/plot_gonzag_Xa.png')

In [74]:
Xm = Xm1[:,:] + Xa[:,:]*float(itt - MG.time[ktm1])

In [75]:
print(float(itt - MG.time[ktm1]))

2821.5599999427795


In [76]:
fig=plt.figure(figsize=[7,5])
plt.pcolormesh(Xm)
plt.colorbar()
plt.savefig('../plots/debug/plot_gonzag_Xm.png')

In [77]:
            [ [j1,i1],[j2,i2],[j3,i3],[j4,i4] ] = BT.SM[jt,:,:]
            [w1, w2, w3, w4]                    = BT.WB[jt,:]


In [78]:
print([j1,i1],[j2,i2],[j3,i3],[j4,i4],w1, w2, w3, w4)

[0, 0] [0, 0] [0, 0] [0, 0] 0.25 0.25 0.25 0.25


In [79]:
Sm = MG.mask[j1,i1] + MG.mask[j2,i2] + MG.mask[j3,i3] + MG.mask[j4,i4]

In [80]:
print(Sm)

4


In [81]:
vssh_m_np[jt] = Xm[j1,i1]

In [83]:
print(j1,i1,vssh_m_np[jt])

0 0 0.49280744791030884
